# Traing the AI

In [24]:
# Import necessary libraries
import os
import librosa
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [25]:
# Define directories for clean and noise files
clean_dir = 'C:/Users/Matt/Documents/Project/CS-M/Experiments/NoiseCancelTests/Datasets/Original/All/(1).wav'
noise_dir = 'C:/Users/Matt/Documents/Project/CS-M/Experiments/NoiseCancelTests/Datasets/Noise/(1).wav'

In [34]:
# Load clean and noise files
clean, sr_clean = librosa.load(clean_dir, sr=None)
noise, sr_noise = librosa.load(noise_dir, sr=None)

# Set hop length to a fixed value
hop_length = 512

# Pad or trim clean and noise files to a multiple of hop length
max_len = max(len(clean), len(noise))
remainder = max_len % hop_length
if remainder != 0:
    clean = librosa.util.pad_center(clean, max_len + hop_length - remainder)
    noise = librosa.util.pad_center(noise, max_len + hop_length - remainder)

# Get STFT of clean and noise files
n_fft = 1024
clean_stft = librosa.stft(clean, n_fft=n_fft, hop_length=hop_length)
noise_stft = librosa.stft(noise, n_fft=n_fft, hop_length=hop_length)

# Reshape input and target data to have the same number of samples
input_data = np.stack((np.abs(clean_stft.T), np.abs(noise_stft.T)), axis=0)
target_data = np.abs(clean_stft.T)[:input_data.shape[1], :]

TypeError: pad_center() takes 1 positional argument but 2 were given

In [27]:
# Define the RNN model
model = Sequential()
model.add(LSTM(256, return_sequences=True, input_shape=(None, 2)))
model.add(Dense(513, activation='linear'))
model.compile(optimizer='adam', loss='mse')

In [28]:
# Train the model
model.fit(input_data, target_data, epochs=50, batch_size=32)

ValueError: Data cardinality is ambiguous:
  x sizes: 2
  y sizes: 1876
Make sure all arrays contain the same number of samples.

In [ ]:
# Export the model
model.save('ANC.h5')

# Testing the AI

In [35]:
# Import necessary libraries
import os
import librosa
import numpy as np
from tensorflow.keras.models import load_model

In [36]:
# Define directories for mixed audio and noise files for testing
mixed_dir = 'MixedAudio.wav'
noise_test_dir = 'C:/Users/Matt/Documents/Project/CS-M/Experiments/NoiseCancelTests/Datasets/Noise/(2).wav'

In [38]:
# Load mixed audio and noise files for testing and get their STFT
mixed, sr_mixed = librosa.load(mixed_dir, sr=None)
noise_test, sr_noise_test = librosa.load(noise_test_dir, sr=None)
mixed_stft = librosa.stft(mixed, n_fft=1024, hop_length=512)
noise_test_stft = librosa.stft(noise_test, n_fft=1024, hop_length=512)

In [39]:
# Create input data for testing the model
test_input = np.stack((np.abs(mixed_stft.T), np.abs(noise_test_stft.T)), axis=0)

In [40]:
# Load the trained RNN model
model = load_model('ANC.h5')

# Use the trained model to predict the clean audio
predicted_clean = model.predict(test_input)

ValueError: in user code:

    File "c:\Users\Matt\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\Matt\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Matt\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\Matt\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1983, in predict_step
        return self(x, training=False)
    File "c:\Users\Matt\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\Matt\AppD

In [ ]:
# Save the predicted clean audio as a WAV file
predicted_clean_audio = librosa.istft(predicted_clean.T, hop_length=512)
librosa.output.write_wav('CleanAudio.wav', predicted_clean_audio, sr_mixed)